In [1]:
import requests
import pandas as pd
import datetime as dt
import numpy as np
from datetime import timedelta
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

BASE_URL = "https://api.the-odds-api.com/v3/odds/?apiKey=d40ba9f6ba3151d8821559bb331101ce&sport=soccer&region=eu"
response = requests.get(BASE_URL)

s= list(response.json().values())[1]
df_list= []

## Pinnacle vs. everyone 

In [153]:
df_list= []

In [154]:

#Level2 - here you'll get columns for each game 
cols= list(s[1].keys())
df= pd.DataFrame(columns= cols)
dat= []
for element in s:
    thing= element.values()
    extra= dict(zip(cols,thing))
    dat.append(extra)
df= df.append(dat)
# Level 3 add the away_team
df['away_team']= 0
for x,i in enumerate(df['teams']):
    if i[0] in df['home_team'][x]:
        df['away_team'][x]= i[1]
    else:
        df['away_team'][x]= i[0]
#Level 4- convert dates 
def unix_conv(unix):
    return(dt.datetime.utcfromtimestamp(unix).strftime('%Y-%m-%dT%H:%M:%SZ'))
    
df['time']= df['commence_time'].map(unix_conv)
df['time']= pd.to_datetime(df['time'], format= "%Y/%m/%d %H:%M")+ timedelta(hours=2)

#Level 5 - add the columns with the odds        
k=[]
for p in df['sites']:
    for i in np.arange(0,len(p)):
        z= list(p[i].values())
        t= z[1]
        k.append(t)
        
a_list= list(set(k))
b_list= ['H','D','A']
oo= ['{}{}'.format(a,b) for b in b_list for a in a_list]
oo.sort()
df[oo]= pd.DataFrame([list(np.repeat(0,len(oo)))], index= df.index)
df[oo]= df[oo].astype('float')

#Level 6 - fill the columns with values 
for name, data in df[oo].iteritems():
    for x,i in enumerate(df['sites']):
        for j in np.arange(0,len(i)):
            imp= list(i[j].values())[1]
            odds= list(list(i[j].values())[3].values())[0]
            if df['home_team'][x]==df['teams'][x][0]:
                order= [0,2,1]
                odds= [odds[i] for i in order]
            elif df['home_team'][x]== df['teams'][x][1]:
                order= [1,2,0]
                odds= [odds[i] for i in order]
            if imp in name and name.endswith('H'):
                df[name][x]= odds[0]
                break
            elif imp in name and name.endswith('D'):
                df[name][x]= odds[1]
                break
            elif imp in name and name.endswith('A'):
                df[name][x]= odds[2]
                break
            else:
                continue
                
# Level 7 Calculate each bookmaker's fair probability [but we only need Pinnacle Sports]

df['ProbsPinnacleH']= 1/df['PinnacleH']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleD']= 1/df['PinnacleD']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleA']= 1/df['PinnacleA']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])


# Level 8 - calculate max odds 
df['MaxOddsH']= df[[x for x in df[oo].columns if x.endswith('H')]].max(axis=1)
df['MaxOddsD']= df[[x for x in df[oo].columns if x.endswith('D')]].max(axis=1)
df['MaxOddsA']= df[[x for x in df[oo].columns if x.endswith('A')]].max(axis=1)

# Level 9 - calculate margin in relation to pinnacle sports probability 
df['MarginH'] = df.ProbsPinnacleH*(df.MaxOddsH-1)-(1-df.ProbsPinnacleH)
df['MarginD'] = df.ProbsPinnacleD*(df.MaxOddsD-1)-(1-df.ProbsPinnacleD)
df['MarginA'] = df.ProbsPinnacleA*(df.MaxOddsA-1)-(1-df.ProbsPinnacleA)

# Level 10 - get max bookies' name
maxBookieH= df[[x for x in df[oo].columns if x.endswith('H')]]
maxBookieD= df[[x for x in df[oo].columns if x.endswith('D')]]
maxBookieA= df[[x for x in df[oo].columns if x.endswith('A')]]
df['MaxBookieH']= 'none'
df['MaxBookieD']= 'none'
df['MaxBookieA']= 'none'


for name, data in maxBookieH.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsH'][i]:
            df['MaxBookieH'][i]= name
            
            
for name, data in maxBookieD.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsD'][i]:
            df['MaxBookieD'][i]= name
            
            
for name, data in maxBookieA.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsA'][i]:
            df['MaxBookieA'][i]= name
            

relevant= ['sport_nice', 'home_team', 'away_team', 'time', 'MaxOddsH','MaxOddsD','MaxOddsA','ProbsPinnacleH', 'ProbsPinnacleD','ProbsPinnacleA', 'MaxBookieH','MaxBookieD', 'MaxBookieA', 'MarginH','MarginD','MarginA']

df_regular= df[relevant].dropna(subset=['MarginH'])
df_regular.name='df_regular'
df_list.append(df_regular)

## Pinnacle vs everyone except Betting Exchanges

In [155]:
#Level2 - here you'll get columns for each game 
cols= list(s[1].keys())
df= pd.DataFrame(columns= cols)
dat= []
for element in s:
    thing= element.values()
    extra= dict(zip(cols,thing))
    dat.append(extra)
df= df.append(dat)
# Level 3 add the away_team
df['away_team']= 0
for x,i in enumerate(df['teams']):
    if i[0] in df['home_team'][x]:
        df['away_team'][x]= i[1]
    else:
        df['away_team'][x]= i[0]
#Level 4- convert dates 
def unix_conv(unix):
    return(dt.datetime.utcfromtimestamp(unix).strftime('%Y-%m-%dT%H:%M:%SZ'))
    
df['time']= df['commence_time'].map(unix_conv)
df['time']= pd.to_datetime(df['time'], format= "%Y/%m/%d %H:%M")+ timedelta(hours=2)

k=[]
for p in df['sites']:
    for i in np.arange(0,len(p)):
        z= list(p[i].values())
        t= z[1]
        k.append(t)

a_list= list(set(k))
b_list= ['H','D','A']
oo= ['{}{}'.format(a,b) for b in b_list for a in a_list]
oo.sort()
df[oo]= pd.DataFrame([list(np.repeat(0,len(oo)))], index= df.index)
df[oo]= df[oo].astype('float')

#Level 6 - fill the columns with values 
for name, data in df[oo].iteritems():
    for x,i in enumerate(df['sites']):
        for j in np.arange(0,len(i)):
            imp= list(i[j].values())[1]
            odds= list(list(i[j].values())[3].values())[0]
            if df['home_team'][x]==df['teams'][x][0]:
                order= [0,2,1]
                odds= [odds[i] for i in order]
            elif df['home_team'][x]== df['teams'][x][1]:
                order= [1,2,0]
                odds= [odds[i] for i in order]
            if imp in name and name.endswith('H'):
                df[name][x]= odds[0]
                break
            elif imp in name and name.endswith('D'):
                #add= list(list(i[j].values())[3].values())[0][1]
                df[name][x]= odds[1]
                break
            elif imp in name and name.endswith('A'):
                #if df['away_team'][x]==df['teams'][x][0]: 
                    #add= list(list(i[j].values())[3].values())[0][0]
                #else:
                   # add= np.round(list(list(i[j].values())[3].values())[0][2],2)
                df[name][x]= odds[2]
                break
            else:
                continue
                
# Level 7 Calculate each bookmaker's fair probability [but we only need Pinnacle Sports]
#df[oo]= pd.DataFrame([list(np.repeat(0,len(oo)))], index= df.index)

probs= ['Probs'+ x for x in oo]

df[probs]= pd.DataFrame([list(np.repeat(0,len(probs)))], index= df.index)
#df[probs]= 0
#df[probs]= df[probs].astype('float')
for j in np.arange(0,len(df)):
    for i in np.arange(0, len(oo), 3):
        home = df[oo].columns[i]
        draw= df[oo].columns[i+1]
        away= df[oo].columns[i+2]
        df['Probs'+ home][j]= 1/df[home][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])
        df['Probs'+draw][j]= 1/df[draw][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])
        df['Probs'+away][j]= 1/df[away][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])

df['ProbsPinnacleH']= 1/df['PinnacleH']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleD']= 1/df['PinnacleD']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleA']= 1/df['PinnacleA']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
# drop exchanges
#df= df.drop(['BetfairH', 'BetfairD', 'BetfairA','MatchbookH','MatchbookD','MatchbookA'], axis= 1, inplace= True)
#df= df.reindex(columns= df.columns)
exchanges= ['BetfairH', 'BetfairD', 'BetfairA','MatchbookH','MatchbookD','MatchbookA','PinnacleH','PinnacleD','PinnacleA']
#df= df[df.columns.difference(exchanges)]

# Level 8 - calculate max odds 
ee= [x for x in oo if x not in exchanges]
df['MaxOddsH']= df[[x for x in df[ee].columns if x.endswith('H')]].max(axis=1)
df['MaxOddsD']= df[[x for x in df[ee].columns if x.endswith('D')]].max(axis=1)
df['MaxOddsA']= df[[x for x in df[ee].columns if x.endswith('A')]].max(axis=1)

# Level 9 - calculate margin in relation to pinnacle sports probability 
df['MarginH'] = df.ProbsPinnacleH*(df.MaxOddsH-1)-(1-df.ProbsPinnacleH)
df['MarginD'] = df.ProbsPinnacleD*(df.MaxOddsD-1)-(1-df.ProbsPinnacleD)
df['MarginA'] = df.ProbsPinnacleA*(df.MaxOddsA-1)-(1-df.ProbsPinnacleA)

# Level 10 - get max bookies' name
maxBookieH= df[[x for x in df[ee].columns if x.endswith('H')]]
maxBookieD= df[[x for x in df[ee].columns if x.endswith('D')]]
maxBookieA= df[[x for x in df[ee].columns if x.endswith('A')]]
df['MaxBookieH']= 'none'
df['MaxBookieD']= 'none'
df['MaxBookieA']= 'none'


for name, data in maxBookieH.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsH'][i]:
            df['MaxBookieH'][i]= name
            
            
for name, data in maxBookieD.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsD'][i]:
            df['MaxBookieD'][i]= name
            
            
for name, data in maxBookieA.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsA'][i]:
            df['MaxBookieA'][i]= name
            

relevant= ['sport_nice', 'home_team', 'away_team', 'time', 'MaxOddsH','MaxOddsD','MaxOddsA','ProbsPinnacleH', 'ProbsPinnacleD','ProbsPinnacleA', 'MaxBookieH','MaxBookieD', 'MaxBookieA', 'MarginH','MarginD','MarginA']

df_without_exchanges= df[relevant].dropna(subset=['MarginH'])
df_without_exchanges.name='df_without_exchanges'
df_list.append(df_without_exchanges)

## Consensus of exchanges and pinnacle vs the rest 

In [156]:
#Level2 - here you'll get columns for each game 
cols= list(s[1].keys())
df= pd.DataFrame(columns= cols)
dat= []
for element in s:
    thing= element.values()
    extra= dict(zip(cols,thing))
    dat.append(extra)
df= df.append(dat)
# Level 3 add the away_team
df['away_team']= 0
for x,i in enumerate(df['teams']):
    if i[0] in df['home_team'][x]:
        df['away_team'][x]= i[1]
    else:
        df['away_team'][x]= i[0]
#Level 4- convert dates 
def unix_conv(unix):
    return(dt.datetime.utcfromtimestamp(unix).strftime('%Y-%m-%dT%H:%M:%SZ'))
    
df['time']= df['commence_time'].map(unix_conv)
df['time']= pd.to_datetime(df['time'], format= "%Y/%m/%d %H:%M")+ timedelta(hours=2)

k=[]
for p in df['sites']:
    for i in np.arange(0,len(p)):
        z= list(p[i].values())
        t= z[1]
        k.append(t)

a_list= list(set(k))
b_list= ['H','D','A']
oo= ['{}{}'.format(a,b) for b in b_list for a in a_list]
oo.sort()
df[oo]= pd.DataFrame([list(np.repeat(0,len(oo)))], index= df.index)
df[oo]= df[oo].astype('float')

#Level 6 - fill the columns with values 
for name, data in df[oo].iteritems():
    for x,i in enumerate(df['sites']):
        for j in np.arange(0,len(i)):
            imp= list(i[j].values())[1]
            odds= list(list(i[j].values())[3].values())[0]
            if df['home_team'][x]==df['teams'][x][0]:
                order= [0,2,1]
                odds= [odds[i] for i in order]
            elif df['home_team'][x]== df['teams'][x][1]:
                order= [1,2,0]
                odds= [odds[i] for i in order]
            if imp in name and name.endswith('H'):
                df[name][x]= odds[0]
                break
            elif imp in name and name.endswith('D'):
                #add= list(list(i[j].values())[3].values())[0][1]
                df[name][x]= odds[1]
                break
            elif imp in name and name.endswith('A'):
                #if df['away_team'][x]==df['teams'][x][0]: 
                    #add= list(list(i[j].values())[3].values())[0][0]
                #else:
                   # add= np.round(list(list(i[j].values())[3].values())[0][2],2)
                df[name][x]= odds[2]
                break
            else:
                continue
                
# Level 7 Calculate each bookmaker's fair probability [but we only need Pinnacle Sports]
#df[oo]= pd.DataFrame([list(np.repeat(0,len(oo)))], index= df.index)

probs= ['Probs'+ x for x in oo]

df[probs]= pd.DataFrame([list(np.repeat(0,len(probs)))], index= df.index).astype('float')
#df[probs]= 0
#df[probs]= df[probs].astype('float')
for j in np.arange(0,len(df)):
    for i in np.arange(0, len(oo), 3):
        home = df[oo].columns[i+2]
        draw= df[oo].columns[i+1]
        away= df[oo].columns[i]
        df['Probs'+ home][j]= 1/df[home][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])
        df['Probs'+draw][j]= 1/df[draw][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])
        df['Probs'+away][j]= 1/df[away][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])

df['ProbsPinnacleH']= 1/df['PinnacleH']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleD']= 1/df['PinnacleD']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleA']= 1/df['PinnacleA']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])

exchanges= ['BetfairH', 'BetfairD', 'BetfairA','MatchbookH','MatchbookD','MatchbookA','PinnacleH','PinnacleD','PinnacleA']
home= ['ProbsBetfairH','ProbsMatchbookH', 'ProbsPinnacleH']
home_ex= [x for x in home if x in df.columns]
draw= ['ProbsBetfairD','ProbsMatchbookD', 'ProbsPinnacleD']
draw_ex= [x for x in draw if x in df.columns]
away= ['ProbsBetfairA','ProbsMatchbookA', 'ProbsPinnacleA']
away_ex= [x for x in away if x in df.columns]

df['ConsProbsH']= df[df[home_ex]>0][home_ex].mean(axis=1)
df['ConsProbsD']= df[df[draw_ex]>0][draw_ex].mean(axis=1)
df['ConsProbsA']= df[df[away_ex]>0][away_ex].mean(axis=1)

# drop exchanges
#df= df.drop(['BetfairH', 'BetfairD', 'BetfairA','MatchbookH','MatchbookD','MatchbookA'], axis= 1, inplace= True)
#df= df.reindex(columns= df.columns)
#df= df[df.columns.difference(exchanges)]

# Level 8 - calculate max odds 
ee= [x for x in oo if x not in exchanges]
df['MaxOddsH']= df[[x for x in df[ee].columns if x.endswith('H')]].max(axis=1)
df['MaxOddsD']= df[[x for x in df[ee].columns if x.endswith('D')]].max(axis=1)
df['MaxOddsA']= df[[x for x in df[ee].columns if x.endswith('A')]].max(axis=1)

# Level 9 - calculate margin in relation to consensus probability 
df['MarginH'] = df.ConsProbsH*(df.MaxOddsH-1)-(1-df.ConsProbsH)
df['MarginD'] = df.ConsProbsD*(df.MaxOddsD-1)-(1-df.ConsProbsD)
df['MarginA'] = df.ConsProbsA*(df.MaxOddsA-1)-(1-df.ConsProbsA)

# Level 10 - get max bookies' name
maxBookieH= df[[x for x in df[ee].columns if x.endswith('H')]]
maxBookieD= df[[x for x in df[ee].columns if x.endswith('D')]]
maxBookieA= df[[x for x in df[ee].columns if x.endswith('A')]]
df['MaxBookieH']= 'none'
df['MaxBookieD']= 'none'
df['MaxBookieA']= 'none'


for name, data in maxBookieH.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsH'][i]:
            df['MaxBookieH'][i]= name
            
            
for name, data in maxBookieD.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsD'][i]:
            df['MaxBookieD'][i]= name
            
            
for name, data in maxBookieA.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsA'][i]:
            df['MaxBookieA'][i]= name
            

relevant= ['sport_nice', 'home_team', 'away_team', 'time', 'MaxOddsH','MaxOddsD','MaxOddsA','ConsProbsH', 'ConsProbsD','ConsProbsA', 'MaxBookieH','MaxBookieD', 'MaxBookieA', 'MarginH','MarginD','MarginA']

df_consensus= df[relevant].dropna(subset=['MarginH'])
df_consensus.name= 'df_consensus'
df_list.append(df_consensus)

## Consensus of exchanges vs the rest including Pinnacle

In [157]:
#Level2 - here you'll get columns for each game 
cols= list(s[1].keys())
df= pd.DataFrame(columns= cols)
dat= []
for element in s:
    thing= element.values()
    extra= dict(zip(cols,thing))
    dat.append(extra)
df= df.append(dat)
# Level 3 add the away_team
df['away_team']= 0
for x,i in enumerate(df['teams']):
    if i[0] in df['home_team'][x]:
        df['away_team'][x]= i[1]
    else:
        df['away_team'][x]= i[0]
#Level 4- convert dates 
def unix_conv(unix):
    return(dt.datetime.utcfromtimestamp(unix).strftime('%Y-%m-%dT%H:%M:%SZ'))
    
df['time']= df['commence_time'].map(unix_conv)
df['time']= pd.to_datetime(df['time'], format= "%Y/%m/%d %H:%M")+ timedelta(hours=2)

k=[]
for p in df['sites']:
    for i in np.arange(0,len(p)):
        z= list(p[i].values())
        t= z[1]
        k.append(t)

a_list= list(set(k))
b_list= ['H','D','A']
oo= ['{}{}'.format(a,b) for b in b_list for a in a_list]
oo.sort()
df[oo]= pd.DataFrame([list(np.repeat(0,len(oo)))], index= df.index)
df[oo]= df[oo].astype('float')

#Level 6 - fill the columns with values 
for name, data in df[oo].iteritems():
    for x,i in enumerate(df['sites']):
        for j in np.arange(0,len(i)):
            imp= list(i[j].values())[1]
            odds= list(list(i[j].values())[3].values())[0]
            if df['home_team'][x]==df['teams'][x][0]:
                order= [0,2,1]
                odds= [odds[i] for i in order]
            elif df['home_team'][x]== df['teams'][x][1]:
                order= [1,2,0]
                odds= [odds[i] for i in order]
            if imp in name and name.endswith('H'):
                df[name][x]= odds[0]
                break
            elif imp in name and name.endswith('D'):
                #add= list(list(i[j].values())[3].values())[0][1]
                df[name][x]= odds[1]
                break
            elif imp in name and name.endswith('A'):
                #if df['away_team'][x]==df['teams'][x][0]: 
                    #add= list(list(i[j].values())[3].values())[0][0]
                #else:
                   # add= np.round(list(list(i[j].values())[3].values())[0][2],2)
                df[name][x]= odds[2]
                break
            else:
                continue
                
# Level 7 Calculate each bookmaker's fair probability [but we only need Pinnacle Sports]
#df[oo]= pd.DataFrame([list(np.repeat(0,len(oo)))], index= df.index)

probs= ['Probs'+ x for x in oo]

df[probs]= pd.DataFrame([list(np.repeat(0,len(probs)))], index= df.index).astype('float')
#df[probs]= 0
#df[probs]= df[probs].astype('float')
for j in np.arange(0,len(df)):
    for i in np.arange(0, len(oo), 3):
        home = df[oo].columns[i+2]
        draw= df[oo].columns[i+1]
        away= df[oo].columns[i]
        df['Probs'+ home][j]= 1/df[home][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])
        df['Probs'+draw][j]= 1/df[draw][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])
        df['Probs'+away][j]= 1/df[away][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])

df['ProbsPinnacleH']= 1/df['PinnacleH']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleD']= 1/df['PinnacleD']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleA']= 1/df['PinnacleA']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])

exchanges= ['BetfairH', 'BetfairD', 'BetfairA','MatchbookH','MatchbookD','MatchbookA']
home= ['ProbsBetfairH','ProbsMatchbookH', 'ProbsPinnacleH']
home_ex= [x for x in home if x in df.columns]
draw= ['ProbsBetfairD','ProbsMatchbookD', 'ProbsPinnacleD']
draw_ex= [x for x in draw if x in df.columns]
away= ['ProbsBetfairA','ProbsMatchbookA', 'ProbsPinnacleA']
away_ex= [x for x in away if x in df.columns]

df['ConsProbsH']= df[df[home_ex]>0][home_ex].mean(axis=1)
df['ConsProbsD']= df[df[draw_ex]>0][draw_ex].mean(axis=1)
df['ConsProbsA']= df[df[away_ex]>0][away_ex].mean(axis=1)

# drop exchanges
#df= df.drop(['BetfairH', 'BetfairD', 'BetfairA','MatchbookH','MatchbookD','MatchbookA'], axis= 1, inplace= True)
#df= df.reindex(columns= df.columns)
#df= df[df.columns.difference(exchanges)]

# Level 8 - calculate max odds 
ee= [x for x in oo if x not in exchanges]
df['MaxOddsH']= df[[x for x in df[ee].columns if x.endswith('H')]].max(axis=1)
df['MaxOddsD']= df[[x for x in df[ee].columns if x.endswith('D')]].max(axis=1)
df['MaxOddsA']= df[[x for x in df[ee].columns if x.endswith('A')]].max(axis=1)

# Level 9 - calculate margin in relation to consensus probability 
df['MarginH'] = df.ConsProbsH*(df.MaxOddsH-1)-(1-df.ConsProbsH)
df['MarginD'] = df.ConsProbsD*(df.MaxOddsD-1)-(1-df.ConsProbsD)
df['MarginA'] = df.ConsProbsA*(df.MaxOddsA-1)-(1-df.ConsProbsA)

# Level 10 - get max bookies' name
maxBookieH= df[[x for x in df[ee].columns if x.endswith('H')]]
maxBookieD= df[[x for x in df[ee].columns if x.endswith('D')]]
maxBookieA= df[[x for x in df[ee].columns if x.endswith('A')]]
df['MaxBookieH']= 'none'
df['MaxBookieD']= 'none'
df['MaxBookieA']= 'none'


for name, data in maxBookieH.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsH'][i]:
            df['MaxBookieH'][i]= name
            
            
for name, data in maxBookieD.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsD'][i]:
            df['MaxBookieD'][i]= name
            
            
for name, data in maxBookieA.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsA'][i]:
            df['MaxBookieA'][i]= name
            

relevant= ['sport_nice', 'home_team', 'away_team', 'time', 'MaxOddsH','MaxOddsD','MaxOddsA','ConsProbsH', 'ConsProbsD','ConsProbsA', 'MaxBookieH','MaxBookieD', 'MaxBookieA', 'MarginH','MarginD','MarginA']

df_consensus2= df[relevant].dropna(subset=['MarginH'])
df_consensus2.name= 'df_consensus2'
df_list.append(df_consensus2)

## Consensus of all vs the best 

In [158]:
#Level2 - here you'll get columns for each game 
cols= list(s[1].keys())
df= pd.DataFrame(columns= cols)
dat= []
for element in s:
    thing= element.values()
    extra= dict(zip(cols,thing))
    dat.append(extra)
df= df.append(dat)
# Level 3 add the away_team
df['away_team']= 0
for x,i in enumerate(df['teams']):
    if i[0] in df['home_team'][x]:
        df['away_team'][x]= i[1]
    else:
        df['away_team'][x]= i[0]
#Level 4- convert dates 
def unix_conv(unix):
    return(dt.datetime.utcfromtimestamp(unix).strftime('%Y-%m-%dT%H:%M:%SZ'))
    
df['time']= df['commence_time'].map(unix_conv)
df['time']= pd.to_datetime(df['time'], format= "%Y/%m/%d %H:%M")+ timedelta(hours=2)

k=[]
for p in df['sites']:
    for i in np.arange(0,len(p)):
        z= list(p[i].values())
        t= z[1]
        k.append(t)

a_list= list(set(k))
b_list= ['H','D','A']
oo= ['{}{}'.format(a,b) for b in b_list for a in a_list]
oo.sort()
df[oo]= pd.DataFrame([list(np.repeat(0,len(oo)))], index= df.index)
df[oo]= df[oo].astype('float')

#Level 6 - fill the columns with values 
for name, data in df[oo].iteritems():
    for x,i in enumerate(df['sites']):
        for j in np.arange(0,len(i)):
            imp= list(i[j].values())[1]
            odds= list(list(i[j].values())[3].values())[0]
            if df['home_team'][x]==df['teams'][x][0]:
                order= [0,2,1]
                odds= [odds[i] for i in order]
            elif df['home_team'][x]== df['teams'][x][1]:
                order= [1,2,0]
                odds= [odds[i] for i in order]
            if imp in name and name.endswith('H'):
                df[name][x]= odds[0]
                break
            elif imp in name and name.endswith('D'):
                #add= list(list(i[j].values())[3].values())[0][1]
                df[name][x]= odds[1]
                break
            elif imp in name and name.endswith('A'):
                #if df['away_team'][x]==df['teams'][x][0]: 
                    #add= list(list(i[j].values())[3].values())[0][0]
                #else:
                   # add= np.round(list(list(i[j].values())[3].values())[0][2],2)
                df[name][x]= odds[2]
                break
            else:
                continue
                
# Level 7 Calculate each bookmaker's fair probability [but we only need Pinnacle Sports]
#df[oo]= pd.DataFrame([list(np.repeat(0,len(oo)))], index= df.index)

probs= ['Probs'+ x for x in oo]

df[probs]= pd.DataFrame([list(np.repeat(0,len(probs)))], index= df.index).astype('float')

for j in np.arange(0,len(df)):
    for i in np.arange(0, len(oo), 3):
        home = df[oo].columns[i+2]
        draw= df[oo].columns[i+1]
        away= df[oo].columns[i]
        df['Probs'+ home][j]= 1/df[home][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])
        df['Probs'+draw][j]= 1/df[draw][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])
        df['Probs'+away][j]= 1/df[away][j]/(1/df[home][j]+1/df[draw][j]+1/df[away][j])

df['ProbsPinnacleH']= 1/df['PinnacleH']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleD']= 1/df['PinnacleD']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])
df['ProbsPinnacleA']= 1/df['PinnacleA']/(1/df['PinnacleH'] + 1/df['PinnacleD'] + 1/df['PinnacleA'])

# Get consensus Probability 
home= [x for x in df[probs].columns if x.endswith('H')]
draw= [x for x in df[probs].columns if x.endswith('D')]
away= [x for x in df[probs].columns if x.endswith('A')]

df['ConsProbsH']= df[df[probs]>0][home].mean(axis=1)
df['ConsProbsD']= df[df[probs]>0][draw].mean(axis=1)
df['ConsProbsA']= df[df[probs]>0][away].mean(axis=1)


# Level 8 - calculate max odds 

df['MaxOddsH']= df[[x for x in df[oo].columns if x.endswith('H')]].max(axis=1)
df['MaxOddsD']= df[[x for x in df[oo].columns if x.endswith('D')]].max(axis=1)
df['MaxOddsA']= df[[x for x in df[oo].columns if x.endswith('A')]].max(axis=1)

# Level 9 - calculate margin in relation to consensus probability 
df['MarginH'] = df.ConsProbsH*(df.MaxOddsH-1)-(1-df.ConsProbsH)
df['MarginD'] = df.ConsProbsD*(df.MaxOddsD-1)-(1-df.ConsProbsD)
df['MarginA'] = df.ConsProbsA*(df.MaxOddsA-1)-(1-df.ConsProbsA)

# Level 10 - get max bookies' name
maxBookieH= df[[x for x in df[oo].columns if x.endswith('H')]]
maxBookieD= df[[x for x in df[oo].columns if x.endswith('D')]]
maxBookieA= df[[x for x in df[oo].columns if x.endswith('A')]]
df['MaxBookieH']= 'none'
df['MaxBookieD']= 'none'
df['MaxBookieA']= 'none'


for name, data in maxBookieH.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsH'][i]:
            df['MaxBookieH'][i]= name
            
            
for name, data in maxBookieD.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsD'][i]:
            df['MaxBookieD'][i]= name
            
            
for name, data in maxBookieA.iteritems():
    for i in np.arange(0, len(df)):
        if data[i] == df['MaxOddsA'][i]:
            df['MaxBookieA'][i]= name
            

relevant= ['sport_nice', 'home_team', 'away_team', 'time', 'MaxOddsH','MaxOddsD','MaxOddsA','ConsProbsH', 'ConsProbsD','ConsProbsA', 'MaxBookieH','MaxBookieD', 'MaxBookieA', 'MarginH','MarginD','MarginA']

df_consensus3= df[relevant].dropna(subset=['MarginH'])
df_consensus3.name= 'df_consensus3'
df_list.append(df_consensus3)

# Add to existing data

In [160]:
olds= []
for x in df_list:
    olds.append(pd.read_csv(str(x.name+'.csv')))

for x in olds:
    x['time']= pd.to_datetime(x['time'])
for i,x in enumerate(df_list):
    name= df_list[i].name
    df_list[i]= df_list[i].append(olds[i]).drop_duplicates().sort_values('time').reset_index(drop= True)
    df_list[i].name= name

In [170]:
#df_list[2].sort_values('time').reset_index(drop= True)
df_list[4].tail(2)


,sport_nice,home_team,away_team,time,MaxOddsH,MaxOddsD,MaxOddsA,ConsProbsH,ConsProbsD,ConsProbsA,MaxBookieH,MaxBookieD,MaxBookieA,MarginH,MarginD,MarginA
20,K League 1,Jeonbuk Hyundai Motors,Seongnam FC,2021-04-18 09:30:00+00:00,1.57,4.12,7.40,0.608069,0.248891,0.143039,Marathon BetH,PinnacleD,BetfairA,-0.045331,0.025432,0.058491
21,J League,Gamba Osaka,Shimizu S Pulse,2021-04-18 10:00:00+00:00,2.28,3.44,3.48,0.433744,0.282086,0.284170,MatchbookH,MatchbookD,MatchbookA,-0.011064,-0.029625,-0.011087


In [166]:
for x in df_list:
    x.to_csv(str(x.name+'.csv'), index= False)